In [3]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import vector
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

# Try to import awkward for vector branch handling
try:
    import awkward as ak
    AWKWARD_AVAILABLE = True
except ImportError:
    AWKWARD_AVAILABLE = False

### 

In [2]:
#Defining functions to read the file

def root_to_dataframe(file_path, tree_name="reco", convert_units=True, verbose=True):
    """
    Convert a ROOT file to a pandas DataFrame with one column per branch.
    
    Parameters:
    -----------
    file_path : str
        Path to the ROOT file
    tree_name : str, default="reco"
        Name of the tree in the ROOT file
    convert_units : bool, default=True
        Whether to convert energy variables from MeV to GeV
    verbose : bool, default=True
        Whether to print progress information
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with events as rows and branches as columns
        Vector branches contain arrays
    """
    
    if verbose:
        print("Converting ROOT file to DataFrame")
        print(f"   File: {file_path}")
        print(f"   Tree: {tree_name}")
    
    # Check if file exists
    if not Path(file_path).exists():
        raise FileNotFoundError(f"ROOT file not found: {file_path}")
    
    # Open ROOT file and extract all branches
    data = {}
    
    with uproot.open(file_path) as file:
        tree = file[tree_name]
        
        if verbose:
            print(f"   Total branches: {len(tree.keys())}")
            print(f"   Total events: {tree.num_entries}")
        
        scalar_count = 0
        vector_count = 0
        
        # Extract all branches
        for branch_name in tree.keys():
            branch = tree[branch_name]
            
            try:
                # Try to read as numpy array (works for scalar branches)
                values = branch.array(library="np")
                
                # Apply unit conversion if requested
                if convert_units and _should_convert_to_gev(branch_name):
                    values = values / 1000.0  # MeV → GeV
                
                data[branch_name] = values
                scalar_count += 1
                
            except ValueError:
                # This is a vector branch, use awkward arrays
                if AWKWARD_AVAILABLE:
                    try:
                        # Get awkward array
                        branch_data = branch.array(library="ak")
                        
                        # Convert to list of numpy arrays
                        arrays = []
                        for i in range(tree.num_entries):
                            arr = ak.to_numpy(branch_data[i])
                            
                            # Apply unit conversion if requested
                            if convert_units and _should_convert_to_gev(branch_name):
                                arr = arr / 1000.0  # MeV → GeV
                            
                            arrays.append(arr)
                        
                        data[branch_name] = arrays
                        vector_count += 1
                        
                        if verbose:
                            avg_multiplicity = np.mean([len(arr) for arr in arrays])
                            print(f"   Vector branch: {branch_name} (avg {avg_multiplicity:.1f} objects/event)")
                            
                    except Exception as e:
                        if verbose:
                            print(f"   Warning: Could not process vector branch {branch_name}: {e}")
                        continue
                else:
                    if verbose:
                        print(f"   Skipped vector branch {branch_name} (awkward not available)")
                    continue
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    if verbose:
        print(f"\nConversion complete!")
        print(f"   DataFrame shape: {df.shape}")
        print(f"   Scalar branches: {scalar_count}")
        print(f"   Vector branches: {vector_count}")
        print(f"   Total columns: {len(df.columns)}")
        print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
        
        if convert_units:
            energy_cols = [col for col in df.columns if _should_convert_to_gev(col)]
            print(f"   Energy variables converted to GeV: {len(energy_cols)} columns")
    
    return df


def root_to_dataframe_essential(file_path, tree_name="reco", convert_units=True, verbose=True):
    """
    Convert a ROOT file to a pandas DataFrame loading only essential branches for physics analysis.
    This function is optimized for speed and memory usage by loading only required branches.
    
    Parameters:
    -----------
    file_path : str
        Path to the ROOT file
    tree_name : str, default="reco"
        Name of the tree in the ROOT file
    convert_units : bool, default=True
        Whether to convert energy variables from MeV to GeV
    verbose : bool, default=True
        Whether to print progress information
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with events as rows and essential branches as columns
        Vector branches contain arrays
    """
    
    # Define essential branches for physics analysis
    essential_branches = [
        'PDFinfo_PDGID1','PDFinfo_PDGID2','PDFinfo_Q','PDFinfo_X1','PDFinfo_X2', 'PDFinfo_XF1', 
        'PDFinfo_XF2', 'el_charge', 'el_eta', 'el_phi', 'jet_eta', 'jet_phi', 'mu_charge', 'mu_eta','mu_phi',
        'TtbarLjetsNu_spanet_down_index_NOSYS', 'TtbarLjetsNu_spanet_had_b_index_NOSYS',
        'TtbarLjetsNu_spanet_had_top_assignment_NOSYS', 'TtbarLjetsNu_spanet_had_top_detection_NOSYS',
        'TtbarLjetsNu_spanet_lep_b_index_NOSYS', 'TtbarLjetsNu_spanet_lep_top_assignment_NOSYS',
        'TtbarLjetsNu_spanet_lep_top_detection_NOSYS', 'TtbarLjetsNu_spanet_reg_nu_eta_NOSYS',
        'TtbarLjetsNu_spanet_reg_nu_px_NOSYS', 'TtbarLjetsNu_spanet_reg_nu_py_NOSYS',
        'TtbarLjetsNu_spanet_reg_nu_pz_NOSYS', 'TtbarLjetsNu_spanet_reg_ttbar_m_NOSYS',
        'TtbarLjetsNu_spanet_up_index_NOSYS', 
        'el_e_NOSYS', 'el_pt_NOSYS', 'jet_e_NOSYS', 'jet_pt_NOSYS', 'mu_e_NOSYS', 'mu_pt_NOSYS',
        'met_met_NOSYS', 'met_phi_NOSYS', 'met_sumet_NOSYS', 'pass_ejets_NOSYS'
    ]
    
    if verbose:
        print("Converting ROOT file to DataFrame (ESSENTIAL BRANCHES ONLY)")
        print(f"   File: {file_path}")
        print(f"   Tree: {tree_name}")
        print(f"   Loading {len(essential_branches)} essential branches")
    
    # Check if file exists
    if not Path(file_path).exists():
        raise FileNotFoundError(f"ROOT file not found: {file_path}")
    
    # Open ROOT file and extract only essential branches
    data = {}
    
    with uproot.open(file_path) as file:
        tree = file[tree_name]
        
        if verbose:
            print(f"   Total available branches: {len(tree.keys())}")
            print(f"   Total events: {tree.num_entries}")
        
        scalar_count = 0
        vector_count = 0
        missing_branches = []
        
        # Extract only essential branches
        for branch_name in essential_branches:
            if branch_name not in tree.keys():
                missing_branches.append(branch_name)
                continue
                
            branch = tree[branch_name]
            
            try:
                # Try to read as numpy array (works for scalar branches)
                values = branch.array(library="np")
                
                # Apply unit conversion if requested
                if convert_units and _should_convert_to_gev(branch_name):
                    values = values / 1000.0  # MeV → GeV
                
                data[branch_name] = values
                scalar_count += 1
                
            except ValueError:
                # This is a vector branch, use awkward arrays
                if AWKWARD_AVAILABLE:
                    try:
                        # Get awkward array
                        branch_data = branch.array(library="ak")
                        
                        # Convert to list of numpy arrays
                        arrays = []
                        for i in range(tree.num_entries):
                            arr = ak.to_numpy(branch_data[i])
                            
                            # Apply unit conversion if requested
                            if convert_units and _should_convert_to_gev(branch_name):
                                arr = arr / 1000.0  # MeV → GeV
                            
                            arrays.append(arr)
                        
                        data[branch_name] = arrays
                        vector_count += 1
                        
                        if verbose:
                            avg_multiplicity = np.mean([len(arr) for arr in arrays])
                            print(f"   Vector branch: {branch_name} (avg {avg_multiplicity:.1f} objects/event)")
                            
                    except Exception as e:
                        if verbose:
                            print(f"   Warning: Could not process vector branch {branch_name}: {e}")
                        continue
                else:
                    if verbose:
                        print(f"   Skipped vector branch {branch_name} (awkward not available)")
                    continue
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    if verbose:
        print(f"\nESSENTIAL conversion complete!")
        print(f"   DataFrame shape: {df.shape}")
        print(f"   Scalar branches loaded: {scalar_count}")
        print(f"   Vector branches loaded: {vector_count}")
        print(f"   Total columns: {len(df.columns)}")
        print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
        
        if missing_branches:
            print(f"   ⚠️ Missing branches: {len(missing_branches)}")
            for branch in missing_branches:
                print(f"     - {branch}")
        
        if convert_units:
            energy_cols = [col for col in df.columns if _should_convert_to_gev(col)]
            print(f"   Energy variables converted to GeV: {len(energy_cols)} columns")
        
        print(f"   🚀 Memory saved by loading only essential branches!")
    
    return df




def _should_convert_to_gev(branch_name):
    """Helper function to determine if a branch should be converted from MeV to GeV"""
    energy_keywords = ['met', '_e_','energy', 'pt', 'mass', 'm_','_nu_px', '_nu_py', '_nu_pz']
    skip_keywords = ['pdf', 'weight', 'number', 'index', 'id', 'channel', 'run', 'event']
    
    has_energy_keyword = any(keyword in branch_name.lower() for keyword in energy_keywords)
    has_skip_keyword = any(keyword in branch_name.lower() for keyword in skip_keywords)
    
    return has_energy_keyword and not has_skip_keyword



In [4]:
class FourVecHandler:
    """
    Class to handle 4-vector creation and manipulation for particle physics analysis.
    This version minimizes for loops and creates all 8 four-vectors in a single efficient function.
    """
    
    def __init__(self):
        """Initialize the FourVecHandler."""
        pass
    
    def create_lepton_4vector(self, event, lepton_type, verbose=False):
        """
        Create a 4-vector for a specified lepton type.
        
        Args:
            event: Single row from DataFrame (pandas Series)
            lepton_type: Type of lepton ('electron' or 'muon')
            verbose: Whether to print detailed information
        
        Returns:
            vector object or None if no lepton found
        """
        # Define column mappings for different lepton types
        lepton_columns = {
            'electron': {
                'pt': 'el_pt_NOSYS',
                'eta': 'el_eta',
                'phi': 'el_phi',
                'energy': 'el_e_NOSYS'
            },
            'muon': {
                'pt': 'mu_pt_NOSYS',
                'eta': 'mu_eta',
                'phi': 'mu_phi',
                'energy': 'mu_e_NOSYS'
            }
        }
        
        if lepton_type not in lepton_columns:
            print(f"❌ Unknown lepton type: {lepton_type}")
            return None
        
        cols = lepton_columns[lepton_type]
        
        # Check if required columns exist
        if not all(col in event.index for col in cols.values()):
            print(f"❌ Missing columns for {lepton_type}")
            return None
        
        # Get lepton data
        pt = event[cols['pt']]
        eta = event[cols['eta']]
        phi = event[cols['phi']]
        energy = event[cols['energy']]
        
        # Handle arrays or single values
        if hasattr(pt, '__len__') and not isinstance(pt, str):
            # If it's an array, take the first lepton
            if len(pt) > 0:
                lepton_vec = vector.obj(
                    pt=float(pt[0]),
                    eta=float(eta[0]),
                    phi=float(phi[0]),
                    energy=float(energy[0])
                )
                if verbose:
                    print(f"Created {lepton_type} 4-vector: pt={lepton_vec.pt:.4f}, eta={lepton_vec.eta:.4f}, phi={lepton_vec.phi:.4f}")
                return lepton_vec
            else:
                print(f"❌ No {lepton_type}s in this event")
                return None
        else:
            # Single value
            lepton_vec = vector.obj(
                pt=float(pt),
                eta=float(eta),
                phi=float(phi),
                energy=float(energy)
            )
            if verbose:
                print(f"Created {lepton_type} 4-vector: pt={lepton_vec.pt:.4f}, eta={lepton_vec.eta:.4f}, phi={lepton_vec.phi:.4f}")
            return lepton_vec

    def create_hadron_4vector(self, event, jet_index=0, verbose=False):
        """
        Create a 4-vector for a hadron jet using the energy directly.
        
        Parameters:
        -----------
        event : pandas.Series
            Event data containing jet information
        jet_index : int
            Index of the jet to use (default: 0)
        verbose : bool
            Whether to print debug information
        
        Returns:
        --------
        vector object or None
        """
        
        # Define jet column names
        jet_columns = {
            'pt': 'jet_pt_NOSYS',
            'eta': 'jet_eta',
            'phi': 'jet_phi',
            'energy': 'jet_e_NOSYS'
        }
        
        # Check if required columns exist
        if not all(col in event.index for col in jet_columns.values()):
            if verbose:
                print(f"Missing required jet columns")
            return None
        
        # Get jet data
        jet_pt = event[jet_columns['pt']]
        jet_eta = event[jet_columns['eta']]
        jet_phi = event[jet_columns['phi']]
        jet_energy = event[jet_columns['energy']]
        
        # Handle arrays or single values
        if hasattr(jet_pt, '__len__') and not isinstance(jet_pt, str):
            if len(jet_pt) > jet_index:
                try:
                    # Create 4-vector
                    jet_vec = vector.obj(
                        pt=float(jet_pt[jet_index]),
                        eta=float(jet_eta[jet_index]),
                        phi=float(jet_phi[jet_index]),
                        energy=float(jet_energy[jet_index])
                    )
                    
                    if verbose:
                        print(f"Created jet {jet_index}: pt={jet_vec.pt:.4f}, eta={jet_vec.eta:.4f}, phi={jet_vec.phi:.4f}")
                    
                    return jet_vec
                    
                except (ValueError, IndexError, TypeError) as e:
                    if verbose:
                        print(f"Error creating jet 4-vector: {e}")
                    return None
            else:
                if verbose:
                    print(f"Jet index {jet_index} not available (only {len(jet_pt)} jets)")
                return None
        else:
            # Single jet value
            if jet_index == 0:
                try:
                    pt_val = float(jet_pt)
                    eta_val = float(jet_eta)
                    phi_val = float(jet_phi)
                    energy_val = float(jet_energy)
                    
                    jet_vec = vector.obj(pt=pt_val, eta=eta_val, phi=phi_val, energy=energy_val)
                    
                    if verbose:
                        print(f"   Created single jet: pt={jet_vec.pt:.3f}, eta={jet_vec.eta:.3f}, mass={jet_vec.mass:.3f}")
                    
                    return jet_vec
                    
                except (ValueError, TypeError) as e:
                    if verbose:
                        print(f"   Error creating single jet: {e}")
                    return None
            else:
                if verbose:
                    print(f"❌ Only single jet available, cannot access index {jet_index}")
                return None

    def create_all_4vectors(self, source_df, verbose=True):
        """
        Function to create all 8 four-vectors
        
        Args:
            source_df: DataFrame containing all required physics data
            verbose: Whether to print detailed information
        
        Returns:
            DataFrame with all 8 four-vector columns and top masses
        """
        
        if verbose:
            print("🚀 Creating all 8 four-vectors using OPTIMIZED approach...")
            print(f"   Processing {len(source_df)} events with minimal for loops")
        
        # Check required columns
        required_cols = {
            'neutrino': ['TtbarLjetsNu_spanet_reg_nu_eta_NOSYS', 'TtbarLjetsNu_spanet_reg_nu_px_NOSYS', 
                        'TtbarLjetsNu_spanet_reg_nu_py_NOSYS', 'TtbarLjetsNu_spanet_reg_nu_pz_NOSYS'],
            'spanet_indices': ['TtbarLjetsNu_spanet_down_index_NOSYS', 'TtbarLjetsNu_spanet_up_index_NOSYS',
                              'TtbarLjetsNu_spanet_had_b_index_NOSYS', 'TtbarLjetsNu_spanet_lep_b_index_NOSYS'],
            'leptons': ['el_eta', 'mu_eta', 'el_pt_NOSYS', 'mu_pt_NOSYS', 'el_phi', 'mu_phi', 
                       'el_e_NOSYS', 'mu_e_NOSYS'],
            'jets': ['jet_pt_NOSYS', 'jet_eta', 'jet_phi', 'jet_e_NOSYS']
        }
        
        # Verify all required columns exist
        missing_cols = []
        for category, cols in required_cols.items():
            missing = [col for col in cols if col not in source_df.columns]
            if missing:
                missing_cols.extend(missing)
        
        if missing_cols:
            print(f"❌ Missing required columns: {missing_cols}")
            return source_df
        
        # ========================================
        # STEP 1: Get neutrino component columns
        # ========================================
        if verbose:
            print("   Step 1: Preparing neutrino component data...")
        
        # Get neutrino columns (will be used inside the loop)
        nu_px_col = 'TtbarLjetsNu_spanet_reg_nu_px_NOSYS'
        nu_py_col = 'TtbarLjetsNu_spanet_reg_nu_py_NOSYS'
        nu_pz_col = 'TtbarLjetsNu_spanet_reg_nu_pz_NOSYS'
        nu_eta_col = 'TtbarLjetsNu_spanet_reg_nu_eta_NOSYS'
        
        # ========================================
        # STEP 2: Pre-allocate arrays for all 8 4-vectors
        # ========================================
        n_events = len(source_df)
        
        # Initialize arrays for all 8 four-vectors
        down_4vecs = np.empty(n_events, dtype=object)
        up_4vecs = np.empty(n_events, dtype=object)
        had_b_4vecs = np.empty(n_events, dtype=object)
        lep_b_4vecs = np.empty(n_events, dtype=object)
        neutrino_4vecs = np.empty(n_events, dtype=object)
        lepton_4vecs = np.empty(n_events, dtype=object)
        had_t_4vecs = np.empty(n_events, dtype=object)
        lep_t_4vecs = np.empty(n_events, dtype=object)
        ttbar_4vecs = np.empty(n_events, dtype=object)

        # Arrays for invariant lepton types, beta
        lepton_types = np.empty(n_events, dtype=object)
        beta = np.empty(n_events, dtype=object)

        # ========================================
        # STEP 3: Single optimized for loop
        # ========================================
        if verbose:
            print("   Step 2: Single optimized loop to create all 8 four-vectors per event...")
        
        # Counters for statistics
        success_counts = {
            'down': 0, 'up': 0, 'had_b': 0, 'lep_b': 0,
            'neutrino': 0, 'lepton': 0, 'had_t': 0, 'lep_t': 0
        }
        
        for i in range(n_events):
            event = source_df.iloc[i]
            
            # ---- Neutrino 4-vector  ----
            nu_px = event[nu_px_col]
            nu_py = event[nu_py_col]
            nu_pz = event[nu_pz_col]
            nu_eta = event[nu_eta_col]
            
            if all(pd.notna(val) for val in [nu_px, nu_py, nu_pz, nu_eta]):
                try:
                    # Convert px, py, pz to pt, phi, energy
                    nu_pt = np.sqrt(float(nu_px)**2 + float(nu_py)**2)
                    nu_phi = np.arctan2(float(nu_py), float(nu_px))
                    nu_energy = np.sqrt(float(nu_px)**2 + float(nu_py)**2 + float(nu_pz)**2)
                    
                    # Create 4-vector using pt, phi, eta, energy
                    neutrino_4vecs[i] = vector.obj(
                        pt=nu_pt,
                        phi=nu_phi,
                        eta=float(nu_eta),
                        energy=nu_energy
                    )
                    success_counts['neutrino'] += 1
                except Exception as e:
                    if verbose and i < 5:  # Only print first few errors
                        print(f"    Warning: Could not create neutrino for event {i}: {e}")
                    neutrino_4vecs[i] = None
            else:
                neutrino_4vecs[i] = None
            
            # ---- Hadron 4-vectors using SpaNET indices ----
            # Down jet
            down_idx = event['TtbarLjetsNu_spanet_down_index_NOSYS']
            if pd.notna(down_idx) and down_idx >= 0:
                down_4vecs[i] = self.create_hadron_4vector(event, jet_index=int(down_idx), verbose=False)
                if down_4vecs[i] is not None:
                    success_counts['down'] += 1
            else:
                down_4vecs[i] = None
            
            # Up jet
            up_idx = event['TtbarLjetsNu_spanet_up_index_NOSYS']
            if pd.notna(up_idx) and up_idx >= 0:
                up_4vecs[i] = self.create_hadron_4vector(event, jet_index=int(up_idx), verbose=False)
                if up_4vecs[i] is not None:
                    success_counts['up'] += 1
            else:
                up_4vecs[i] = None
            
            # Hadronic b jet
            had_b_idx = event['TtbarLjetsNu_spanet_had_b_index_NOSYS']
            if pd.notna(had_b_idx) and had_b_idx >= 0:
                had_b_4vecs[i] = self.create_hadron_4vector(event, jet_index=int(had_b_idx), verbose=False)
                if had_b_4vecs[i] is not None:
                    success_counts['had_b'] += 1
            else:
                had_b_4vecs[i] = None
            
            # Leptonic b jet
            lep_b_idx = event['TtbarLjetsNu_spanet_lep_b_index_NOSYS']
            if pd.notna(lep_b_idx) and lep_b_idx >= 0:
                lep_b_4vecs[i] = self.create_hadron_4vector(event, jet_index=int(lep_b_idx), verbose=False)
                if lep_b_4vecs[i] is not None:
                    success_counts['lep_b'] += 1
            else:
                lep_b_4vecs[i] = None
            
            # ---- Lepton 4-vector
            if event['pass_ejets_NOSYS'] == 1:
                lepton_4vecs[i] = self.create_lepton_4vector(event, 'electron', verbose=False)
                lepton_types[i] = 'electron'
            else:
                lepton_4vecs[i] = self.create_lepton_4vector(event, 'muon', verbose=False)
                lepton_types[i] = 'muon'
            
            if lepton_4vecs[i] is not None:
                success_counts['lepton'] += 1
            
            # ---- Top quark 4-vectors and observables ----
            # Hadronic top
            if (had_b_4vecs[i] is not None and up_4vecs[i] is not None and down_4vecs[i] is not None):
                try:
                    had_t_4vecs[i] = had_b_4vecs[i] + up_4vecs[i] + down_4vecs[i]
                    success_counts['had_t'] += 1
                except:
                    had_t_4vecs[i] = None
            else:
                had_t_4vecs[i] = None
            
            # Leptonic top
            if (lep_b_4vecs[i] is not None and lepton_4vecs[i] is not None and neutrino_4vecs[i] is not None):
                try:
                    lep_t_4vecs[i] = lep_b_4vecs[i] + lepton_4vecs[i] + neutrino_4vecs[i]
                    success_counts['lep_t'] += 1
                except:
                    lep_t_4vecs[i] = None
            else:
                lep_t_4vecs[i] = None
            
            #ttbar 4-vector
            if (had_t_4vecs[i] is not None and lep_t_4vecs[i] is not None):
                try:
                    ttbar_4vecs[i] = had_t_4vecs[i] + lep_t_4vecs[i]
                except:
                    ttbar_4vecs[i] = None
            else:
                ttbar_4vecs[i] = None
                
            #Beta calculation
            if ttbar_4vecs[i] is not None:
                beta[i] = ttbar_4vecs[i].beta
            else:
                beta[i] = None

            # Progress indicator for large datasets
            if verbose and i > 0 and i % 100 == 0:
                print(f"   Processed {i}/{n_events} events ({i/n_events*100:.1f}%)")
        
        # ========================================
        # STEP 4: Add all columns to DataFrame at once
        # ========================================
        if verbose:
            print("   Step 3: Adding all columns to DataFrame...")
        
        result_df = source_df.copy()
        
        # Add all 8 four-vector columns
        result_df['down_4vec'] = down_4vecs
        result_df['up_4vec'] = up_4vecs
        result_df['had_b_4vec'] = had_b_4vecs
        result_df['lep_b_4vec'] = lep_b_4vecs
        result_df['neutrino_4vec'] = neutrino_4vecs
        result_df['lepton_4vec'] = lepton_4vecs
        result_df['had_t'] = had_t_4vecs
        result_df['lep_t'] = lep_t_4vecs
        
        
        # Add auxiliary columns
        result_df['lepton_type'] = lepton_types
        result_df['ttbar_4vec'] = ttbar_4vecs
        result_df['beta'] = beta
                
        if verbose:
            print(f"\n✅ OPTIMIZED 4-vector creation completed!")
            print(f"   Final DataFrame shape: {result_df.shape}")
            print(f"   Added {8} four-vector columns + {7} auxiliary columns")
            
            print(f"\n📊 Success rates:")
            for vec_type, count in success_counts.items():
                percentage = count / n_events * 100
                print(f"   • {vec_type}: {count}/{n_events} ({percentage:.1f}%)")
            
            # Lepton breakdown
            electron_count = sum(1 for ltype in lepton_types if ltype == 'electron')
            muon_count = sum(1 for ltype in lepton_types if ltype == 'muon')
            print(f"   • Leptons: {electron_count} electrons, {muon_count} muons")
            
            print(f"\n⚡ Performance: Single loop processed {n_events} events efficiently!")
        
        return result_df


### ##############

In [4]:
# Define file path 
root_file_path =  r"/data/dust/group/atlas/ttreco/mc202507/user.kbehr.410470.PhPy8EG.DAOD_PHYS.e6337_s3681_r13145_r13146_p6697.250627-v1_output/user.kbehr.45470290._000001.output.root" 

# Check if file exists
if not Path(root_file_path).exists():
    print(f"❌ ROOT file not found: {root_file_path}")
    print("Please update the file path in the cell above.")
else:
    print(f"ROOT file found: {root_file_path}")
    
    # Load ROOT file to DataFrame
    print("\n🔄 Loading ROOT file...")
    start_time = time.time()
    
    df = root_to_dataframe_essential(
        file_path=root_file_path,
        tree_name="reco",
        convert_units=True,
        verbose=False
    )
    
    load_time = time.time() - start_time
    print(f"\n Loading completed in {load_time:.2f} seconds")
    print(f" DataFrame ready with {len(df)} events and {len(df.columns)} columns")

ROOT file found: /data/dust/group/atlas/ttreco/mc202507/user.kbehr.410470.PhPy8EG.DAOD_PHYS.e6337_s3681_r13145_r13146_p6697.250627-v1_output/user.kbehr.45470290._000001.output.root

🔄 Loading ROOT file...

 Loading completed in 64.58 seconds
 DataFrame ready with 911148 events and 38 columns


In [5]:
#Save the dataframe to pickle file to avoid recomputing everytime
df.to_pickle('df.pkl')

### ###################

In [3]:
# Load DataFrame from pickle file
df_loaded = pd.read_pickle('df.pkl')

In [7]:
# Initialize the optimized FourVecHandler
handler = FourVecHandler()

print("🚀 Starting OPTIMIZED 4-vector creation...")
print("   This will create all 8 four-vectors with minimal for loops!")

# Measure performance
start_time = time.time()

# Create all 8 four-vectors using the optimized function
df_with_4vecs = handler.create_all_4vectors(source_df=df_loaded,verbose=False)

creation_time = time.time() - start_time

print(f"\n⚡ OPTIMIZED creation completed in {creation_time:.2f} seconds")

🚀 Starting OPTIMIZED 4-vector creation...
   This will create all 8 four-vectors with minimal for loops!

⚡ OPTIMIZED creation completed in 308.51 seconds


NameError: name 'df' is not defined

In [8]:
print(f"📊 Performance: {len(df_with_4vecs)/creation_time:.1f} events per second")
print(f"🎯 Created {8} four-vectors + auxiliary columns for {len(df_with_4vecs)} events")

📊 Performance: 2953.4 events per second
🎯 Created 8 four-vectors + auxiliary columns for 911148 events


In [70]:
#Save the dataframe to pickle file to avoid recomputing everytime
df_with_4vecs.to_pickle('df_with_4vecs.pkl')

### 

In [4]:
# Load DataFrame from pickle file
df_with_4vecs = pd.read_pickle('df_with_4vecs.pkl')

In [5]:
df_with_4vecs

,PDFinfo_PDGID1,PDFinfo_PDGID2,PDFinfo_Q,PDFinfo_X1,PDFinfo_X2,PDFinfo_XF1,PDFinfo_XF2,el_charge,el_eta,el_phi,...,up_4vec,had_b_4vec,lep_b_4vec,neutrino_4vec,lepton_4vec,had_t,lep_t,lepton_type,ttbar_4vec,beta
0,21,21,175.477707,0.018292,0.043399,5.279873,2.428524,[],[],[],...,"MomentumObject4D(pt=35.376197814941406, phi=-2...","MomentumObject4D(pt=49.75637435913086, phi=1.3...","MomentumObject4D(pt=80.4171142578125, phi=0.30...","MomentumObject4D(pt=52.23156518116804, phi=2.9...","MomentumObject4D(pt=36.034976959228516, phi=-1...","MomentumObject4D(pt=12.971276624613935, phi=-2...","MomentumObject4D(pt=18.748104745478543, phi=0....",muon,"MomentumObject4D(pt=12.382046348888538, phi=-0...",0.39923
1,21,21,199.347641,0.033969,0.072345,3.081088,1.338084,[-1.0],[0.019284088],[0.8802953],...,"MomentumObject4D(pt=31.418272018432617, phi=-1...","MomentumObject4D(pt=118.49435424804688, phi=2....","MomentumObject4D(pt=62.8841552734375, phi=-0.5...","MomentumObject4D(pt=11.29231493910977, phi=-1....","MomentumObject4D(pt=61.640541076660156, phi=0....","MomentumObject4D(pt=84.08818596013975, phi=2.9...","MomentumObject4D(pt=87.289789982584, phi=0.021...",electron,"MomentumObject4D(pt=17.57234409909676, phi=1.3...",0.490863
2,21,21,175.820526,0.212367,0.054992,0.210094,1.879462,[],[],[],...,"MomentumObject4D(pt=135.8825225830078, phi=-1....","MomentumObject4D(pt=61.62810516357422, phi=2.9...","MomentumObject4D(pt=109.79779815673828, phi=-2...","MomentumObject4D(pt=41.1291942560704, phi=-3.0...","MomentumObject4D(pt=39.138710021972656, phi=-1...","MomentumObject4D(pt=217.0454750825755, phi=-2....","MomentumObject4D(pt=148.61508511282366, phi=-2...",muon,"MomentumObject4D(pt=361.7784100537605, phi=-2....",0.890895
3,21,21,184.677887,0.050892,0.016144,2.043449,5.821816,[1.0],[1.0029755],[3.1009815],...,"MomentumObject4D(pt=65.02371978759766, phi=-0....","MomentumObject4D(pt=62.58217239379883, phi=1.3...","MomentumObject4D(pt=35.11328887939453, phi=0.9...","MomentumObject4D(pt=35.821755863971255, phi=-2...","MomentumObject4D(pt=33.15223693847656, phi=3.1...","MomentumObject4D(pt=28.455146146153126, phi=-0...","MomentumObject4D(pt=27.879776033905735, phi=-3...",electron,"MomentumObject4D(pt=10.29182948559227, phi=-1....",0.533994
4,21,21,182.340729,0.074167,0.014212,1.305422,6.411271,[],[],[],...,"MomentumObject4D(pt=83.33775329589844, phi=1.8...","MomentumObject4D(pt=31.728225708007812, phi=2....","MomentumObject4D(pt=53.85918045043945, phi=-0....","MomentumObject4D(pt=46.45360132027916, phi=-0....","MomentumObject4D(pt=36.46967315673828, phi=-2....","MomentumObject4D(pt=138.24381757730634, phi=2....","MomentumObject4D(pt=96.60617493126811, phi=-1....",muon,"MomentumObject4D(pt=43.39289860174294, phi=2.3...",0.435009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911143,21,21,254.847504,0.039080,0.050805,2.657006,2.007821,[],[],[],...,"MomentumObject4D(pt=159.6160125732422, phi=-0....","MomentumObject4D(pt=32.94810485839844, phi=0.4...","MomentumObject4D(pt=38.77629852294922, phi=0.4...","MomentumObject4D(pt=127.08344446308541, phi=2....","MomentumObject4D(pt=61.388023376464844, phi=-2...","MomentumObject4D(pt=246.90421780824477, phi=-0...","MomentumObject4D(pt=142.46301799000602, phi=2....",muon,"MomentumObject4D(pt=104.56788678528957, phi=-0...",0.259679
911144,21,21,231.103027,0.064083,0.019838,1.538785,4.940284,[],[],[],...,"MomentumObject4D(pt=84.85799407958984, phi=-3....","MomentumObject4D(pt=34.719581604003906, phi=-1...","MomentumObject4D(pt=80.201416015625, phi=-0.84...","MomentumObject4D(pt=20.544440618901067, phi=2....","MomentumObject4D(pt=95.40513610839844, phi=0.5...","MomentumObject4D(pt=85.5972095299252, phi=3.10...","MomentumObject4D(pt=125.83200351415749, phi=0....",muon,"MomentumObject4D(pt=40.94928754059125, phi=0.1...",0.673129
911145,21,21,188.113983,0.047282,0.017741,2.210978,5.409729,[1.0],[0.29584903],[1.6508919],...,"MomentumObject4D(pt=50.544708251953125, phi=-1...","Momen